In [1]:
# Install required packages
!pip install linformer
!pip install vit_pytorch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from torchvision import transforms, datasets
from linformer import Linformer
from vit_pytorch.efficient import ViT
from tqdm import tqdm
import numpy as np
from sklearn.metrics import confusion_matrix

# Define paths
train_dir = '/content/drive/MyDrive/dataset/training'
val_dir = '/content/drive/MyDrive/dataset/test'
test_dir = '/content/drive/MyDrive/dataset/validation'

# Hyperparameters
IMG_SIZE = 224  # Example image size
batch_size = 32
epochs = 10
patch_size = 16
num_classes = 2
lr = 0.001
gamma = 0.7
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define transformations
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load datasets
train_ds = datasets.ImageFolder(train_dir, transform=transform_train)
valid_ds = datasets.ImageFolder(val_dir, transform=transform)
test_ds = datasets.ImageFolder(test_dir, transform=transform)

# Function to rebalance dataset
def rebalance_dataset(dataset):
    class_indices = [np.where(np.array(dataset.targets) == i)[0] for i in range(len(dataset.classes))]
    min_class_len = min(len(class_indices[0]), len(class_indices[1]))
    balanced_indices = np.concatenate([class_indices[i][:min_class_len] for i in range(len(dataset.classes))])
    return torch.utils.data.Subset(dataset, balanced_indices)

# Rebalance validation and test sets
valid_ds_balanced = rebalance_dataset(valid_ds)
test_ds_balanced = rebalance_dataset(test_ds)

# DataLoaders with balanced datasets
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_ds_balanced, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(test_ds_balanced, batch_size=batch_size, shuffle=False, num_workers=0)

# Calculate class weights
class_counts = np.bincount(train_ds.targets)
class_weights = 1. / class_counts
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Define loss function with weights
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Linear Transformer
efficient_transformer = Linformer(dim=128, seq_len=(IMG_SIZE // patch_size) ** 2 + 1, depth=12, heads=8, k=64)

# Vision Transformer Model
model = ViT(
    dim=128,
    image_size=IMG_SIZE,
    patch_size=patch_size,
    num_classes=num_classes,
    transformer=efficient_transformer,
    channels=3
).to(device)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

# Learning rate scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

# Training loop
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    epoch_accuracy = 0
    for data, label in tqdm(train_loader):
        data, label = data.to(device), label.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss / len(train_loader)

    model.eval()
    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in valid_loader:
            data, label = data.to(device), label.to(device)
            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(valid_loader)
            epoch_val_loss += val_loss / len(valid_loader)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}, Val Loss: {epoch_val_loss:.4f}, Val Accuracy: {epoch_val_accuracy:.4f}")

# Save the model
PATH = f"epochs_{epochs}img{IMG_SIZE}patch{patch_size}lr{lr}.pt"
torch.save(model.state_dict(), PATH)

# Load saved model
efficient_transformer = Linformer(dim=128, seq_len=(IMG_SIZE // patch_size) ** 2 + 1, depth=12, heads=8, k=64)
model = ViT(image_size=IMG_SIZE, patch_size=patch_size, num_classes=num_classes, dim=128, transformer=efficient_transformer, channels=3).to(device)
model.load_state_dict(torch.load(PATH))

# Function to calculate overall accuracy
def overall_accuracy(model, test_loader, criterion):
    model.eval()
    y_proba = []
    y_truth = []
    test_loss = 0
    total = 0
    correct = 0
    with torch.no_grad():
        for data, label in tqdm(test_loader):
            data, label = data.to(device), label.to(device)
            output = model(data)
            test_loss += criterion(output, label.long()).item()
            for index, i in enumerate(output):
                y_proba.append(i[1].item())
                y_truth.append(label[index].item())
                if torch.argmax(i) == label[index]:
                    correct += 1
                total += 1
    accuracy = correct / total
    y_proba_out = np.array(y_proba)
    y_truth_out = np.array(y_truth)
    return test_loss, accuracy, y_proba_out, y_truth_out

# Evaluate model on test data
loss, acc, y_proba, y_truth = overall_accuracy(model, test_loader, criterion)

print(f"Test Accuracy: {acc:.4f}")

# Plot confusion matrix
cm = confusion_matrix(y_truth, np.argmax(y_proba.reshape(-1, 1), axis=1))
print(cm)


 27%|██▋       | 10/37 [01:51<04:41, 10.42s/it]